In [1]:
import numpy as np
import face_detection
import os, python_speech_features
import scipy.io.wavfile as wav
import pandas as pd
import torch.nn.functional as F
from talkNet import talkNet
from dataset import MyDataset
from torch.utils.data import DataLoader
import tqdm
import torch
import cv2
import tools
import subprocess
detector = face_detection.build_detector(
"DSFDDetector", confidence_threshold=.5, nms_iou_threshold=.3)

c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\jmmol\miniconda3\envs\alc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
inputVideo =  "C:/Users/jmmol/Desktop/LIP-RTVE/MP4s/speaker295/speaker295_0001.mp4"
#inputVideo = "dos_personas_hablando_por_turnos.mp4"
videoDuration = tools.checkVideoDuration(inputVideo)
tools.splitVideo(inputVideo,2,videoDuration)

In [3]:
#Carga vídeo
res, facePos = tools.saveFaceCrops(inputVideo,detector)
res[0].shape

(112, 112)

In [4]:
# AUDIO PROCESSING
audioPath = tools.convert_video_to_audio_ffmpeg(inputVideo)
_,sig = wav.read(audioPath)#r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker042_0063.wav") #
# _,sig = wav.read(r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav") 
# audioPath = r"C:\Users\jmmol\Desktop\COSAS V7\TFM\speaker000_0000.wav"
audio = python_speech_features.mfcc(sig, 16000, numcep = 13, winlen = 0.025, winstep = 0.010) #ASUME VIDEO A 25 Y AUDIO A 100, MODIFICAR


#TEST
#audio = torch.FloatTensor(audio[:180]).unsqueeze(0)
#video = torch.FloatTensor(res[:45]).unsqueeze(0)


In [5]:
model = talkNet()
model.load_state_dict(torch.load("./exps/exp2/model/model_0002.model"))

05-26 15:37:30 Model para number = 15.01


<All keys matched successfully>

In [6]:
videoDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/npz"
audioDir = "C:/Users/jmmol/Desktop/COSAS V7/TFM/mfccs"
datasetTrain = MyDataset(51,videoDir,audioDir,"testSamples.csv")
item = datasetTrain.__getitem__(4309)
valLoader = DataLoader(dataset=datasetTrain,shuffle=False,batch_size=32,num_workers=14) #Cambiar num_workers

25 100


In [9]:
# correctFrames, totalFrames = 0, 0
# correctSamples, totalSamples = 0, 0
# totalPreds = []
# for num, (audioFeature, visualFeature, labels) in enumerate(tqdm.tqdm(valLoader)):
#     with torch.no_grad():    
#         output = model((audioFeature,visualFeature))
#         labels = labels.cuda()
#         batchPreds = torch.reshape(output[1], labels.shape)
#         #Precision a nivel de video
#         videoPreds = torch.mode(batchPreds,dim=1)[0]
#         totalPreds.extend(videoPreds.detach().cpu().numpy().astype(int))
#         labelMode = torch.mode(labels,dim=1)[0]
#         correctSamples += (videoPreds == labelMode).sum().float()
#         totalSamples += len(labels)
#         # Precision a nivel de frame
#         labels = labels.reshape((-1))
#         correctFrames += (output[1] == labels).sum().float()
#         totalFrames += len(labels) 
# #7865
# print("Frame acc:", correctFrames/totalFrames)
# print("Sample acc:", correctSamples/totalSamples)
center = 47
iAudio = tools.padAudio(audio,1,center,51,len(res))
iVideo = tools.padVideo(res,center,51)
print(iAudio[40],item[0][40])
print("Audios iguales?",(iAudio == item[0]).sum(),iAudio.shape[0]*iAudio.shape[1])
print("Videos iguales?",(iVideo == item[1]).sum(),iVideo.shape[0]*112*112)
print(audio[0],item[0][0])

tensor([ 16.0407,   5.1092,  -7.4174,  26.3697, -21.4168, -34.9214, -26.8578,
        -27.1986, -11.6547,  -2.2427, -25.6342, -12.2112,   4.5576]) tensor([ 16.0407,   5.1092,  -7.4174,  26.3697, -21.4168, -34.9214, -26.8578,
        -27.1986, -11.6547,  -2.2427, -25.6342, -12.2112,   4.5576])
Audios iguales? tensor(2652) 2652
Videos iguales? tensor(269827) 639744
[ 13.54920361  -1.19163837   0.22335959   7.48522507 -34.04939913
 -13.72476479   3.78442819 -15.92853244 -10.21488174  -3.14354969
  -2.80750994   6.02800724  -7.13382001] tensor([ 11.6267,   6.2778,   2.7260,  -5.3083,  -9.2899,  -7.2238,  -8.3037,
         -3.4917,  -6.4616, -10.6301, -17.5955,  -8.8042,  -3.2956])


In [ ]:
predArray = []
for i in range(len(res)):
    print("FRAME Nº ",i)
    center = i
    iAudio = tools.padAudio(audio,1,center,51,len(res)).unsqueeze(0)
    iVideo = tools.padVideo(res,center,51).unsqueeze(0)
    #print("audio shape: ", audio.shape)
    #print("video shape: ", video.shape)
    # iAudio = item[0].unsqueeze(0)
    # iVideo = item[1].unsqueeze(0)
    output = model((iAudio,iVideo))
    #output = model((audio,video))
    #print(output)
    print("Pred ->",torch.mode(output[1])[0])
    print("Prob Pos ->",torch.mean(output[1]))
    predArray.append(torch.mean(output[1]).detach().cpu().numpy())

In [ ]:
def createVideo(imgFrames,audioPath,width,height):
    video = cv2.VideoWriter("output.mp4", 0, 25, (width,height))

    for image in imgFrames:
        video.write(image)

    cv2.destroyAllWindows()
    video.release()
    print(' '.join(["ffmpeg","-y","-i",os.getcwd()+"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+"/output2.mp4"]))
    res = subprocess.check_output(["ffmpeg","-y","-i",os.getcwd()+f"/output.mp4","-i",audioPath,"-map","0:v","-map","1:a", "-c:v", "copy", "-shortest", os.getcwd()+f"/outputs/{inputVideo[-19:]}"])
    os.remove(os.getcwd()+f"/output.mp4")
    
    print(res)

In [ ]:
i=0
cap = cv2.VideoCapture(inputVideo)
videoImages = []
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(predArray)
while i < len(predArray):
    ret, image = cap.read()
    greenValue = int(255*predArray[i])
    redValue = 255-greenValue
    color = (0,greenValue,redValue)
    #print(color)
    
    xmin,ymin,xmax,ymax = facePos[i]
    image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax),color , 1)
    cv2.putText(image, str(predArray[i]), (xmin, ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (36,255,12), 1)
    videoImages.append(image)

    # cv2.imshow("My Video", image)
    # cv2.waitKey(0)
    i+=1
    if ret == False:
        break

cap.release()
cv2.destroyAllWindows()

createVideo(videoImages,audioPath,width,height)

In [ ]:
#print(totalPreds)
dfPreds = pd.DataFrame(totalPreds,columns=["Prediction"])
dfSamples = pd.read_csv("testSamples.csv")
dfRes = dfSamples.join(dfPreds)
print(dfRes)
dfRes.to_csv("testPreds5.csv")

In [ ]:
dfTrain = pd.read_csv("trainSamples.csv")

In [ ]:
dfTrain.join(dfSamples,on="video")

In [ ]:
createVideo(res,audioPath,112,112)